In [7]:
import numpy as np
import pandas as pd
#import patsy as pt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy as pt


from IPython.display import HTML
from ipywidgets import interact
import ipywidgets as widgets
import copy
import warnings
warnings.filterwarnings('ignore')

from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import GaussianNB
from sklearn import datasets
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import linear_model as lm
import statsmodels.api as sm
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.svm import LinearSVC, SVC

# Import classes from scikit-learn for logistic regression, LDA, QDA, and KNN classification
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import SelectKBest, chi2, f_classif

%matplotlib inline

### In this problem, you will develop a model to predict whether income exceeds $\frac{50K}{yr}$ based on census data.

---

| name  | favorite fruit | car   |
|-------|----------------|-------|
| joey  | apple          | tahoe |
| dan   | banana         | lexus |
| ebony | orange         | bmw   |

**a) Use the code in Blackboard to create the adult data set.**

In [4]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
adult = fetch_ucirepo(id=2) 
  
# data (as pandas dataframes) 
X = adult.data.features 
y = adult.data.targets 
  
# metadata 
print(adult.metadata) 
  
# variable information 
print(adult.variables) 


{'uci_id': 2, 'name': 'Adult', 'repository_url': 'https://archive.ics.uci.edu/dataset/2/adult', 'data_url': 'https://archive.ics.uci.edu/static/public/2/data.csv', 'abstract': 'Predict whether income exceeds $50K/yr based on census data. Also known as "Census Income" dataset. ', 'area': 'Social Science', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 48842, 'num_features': 14, 'feature_types': ['Categorical', 'Integer'], 'demographics': ['Age', 'Income', 'Education Level', 'Other', 'Race', 'Sex'], 'target_col': ['income'], 'index_col': None, 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 1996, 'last_updated': 'Mon Aug 07 2023', 'dataset_doi': '10.24432/C5XW20', 'creators': ['Barry Becker', 'Ronny Kohavi'], 'intro_paper': None, 'additional_info': {'summary': 'Extraction was done by Barry Becker from the 1994 Census database.  A set of reasonably clean records was extracted using the following conditions: ((AAG

Now, let's look at the dependent variables

In [8]:
X.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba


In [11]:
X['occupation'].value_counts()

occupation
Prof-specialty       6172
Craft-repair         6112
Exec-managerial      6086
Adm-clerical         5611
Sales                5504
Other-service        4923
Machine-op-inspct    3022
Transport-moving     2355
Handlers-cleaners    2072
?                    1843
Farming-fishing      1490
Tech-support         1446
Protective-serv       983
Priv-house-serv       242
Armed-Forces           15
Name: count, dtype: int64

In [5]:
X.shape

(48842, 14)

In [6]:
y.value_counts()

income
<=50K     24720
<=50K.    12435
>50K       7841
>50K.      3846
Name: count, dtype: int64

In [13]:
y['income']=y.loc[:,'income'].replace(['>50K.'],'>50K')
y['income']=y.loc[:,'income'].replace(['<=50K.'],'<=50K')
y.value_counts()

income
<=50K     37155
>50K      11687
Name: count, dtype: int64

In [25]:
X=pd.get_dummies(X,drop_first=True)
X.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_Federal-gov,workclass_Local-gov,workclass_Never-worked,workclass_Private,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,39,77516,13,2174,0,40,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
1,50,83311,13,0,0,13,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,38,215646,9,0,0,40,False,False,False,True,...,False,False,False,False,False,False,False,True,False,False
3,53,234721,7,0,0,40,False,False,False,True,...,False,False,False,False,False,False,False,True,False,False
4,28,338409,13,0,0,40,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False


In [26]:
f = 'income ~ 1 + ' + ' + '.join(X.columns.drop(['fnlwgt','education-num']))
f

'income ~ 1 + age + capital-gain + capital-loss + hours-per-week + workclass_Federal-gov + workclass_Local-gov + workclass_Never-worked + workclass_Private + workclass_Self-emp-inc + workclass_Self-emp-not-inc + workclass_State-gov + workclass_Without-pay + education_11th + education_12th + education_1st-4th + education_5th-6th + education_7th-8th + education_9th + education_Assoc-acdm + education_Assoc-voc + education_Bachelors + education_Doctorate + education_HS-grad + education_Masters + education_Preschool + education_Prof-school + education_Some-college + marital-status_Married-AF-spouse + marital-status_Married-civ-spouse + marital-status_Married-spouse-absent + marital-status_Never-married + marital-status_Separated + marital-status_Widowed + occupation_Adm-clerical + occupation_Armed-Forces + occupation_Craft-repair + occupation_Exec-managerial + occupation_Farming-fishing + occupation_Handlers-cleaners + occupation_Machine-op-inspct + occupation_Other-service + occupation_Pri

**b) Explore the data graphically in order to investigate the association between income
and the other features. Which of the other features seem most likely to be useful in
predicting income? Scatterplots and boxplots may be useful tools to answer this
question. Describe your findings.**

**c) Split the data into an 80% training set and a 20% test set. Set the seed at 1303.**

In [34]:
train=np.random.random(size=len(y))>0.2
train.sum()/train.shape[0]

0.7994144383931862

**d) Perform LDA on the training data in order to predict income using the variables that
seemed most associated with income in (b). What is the test error of the model
obtained?**

In [35]:
# Fit the LDA model using the training set
lda_clf = LinearDiscriminantAnalysis()
lda_clf.fit(X[train], y[train])

LinearDiscriminantAnalysis()

In [36]:
lda_pred = lda_clf.predict(X[~train])
pd.DataFrame(confusion_matrix(y[~train], lda_pred), 
             columns = ["<=50K", ">50K"], index = ["<=50K", ">50K"])

,<=50K,>50K
<=50K,6966,523
>50K,982,1326


In [39]:
((6966+1326)/(6966+523+982+1326))

0.8463815453710319

In [41]:
lda_clf.score(X[~train], y[~train])

0.15361845462896806

**e) Perform QDA on the training data in order to predict income using the variables that
seemed most associated with income in (b). What is the test error of the model
obtained?**

In [42]:
# Fit the QDA model using the training set
qda_clf = QuadraticDiscriminantAnalysis()
qda_clf.fit(X[train], y[train])

QuadraticDiscriminantAnalysis()

In [43]:
qda_pred = qda_clf.predict(X[~train])
pd.DataFrame(confusion_matrix(y[~train], qda_pred), 
             columns = ["<=50K", ">50K"], index = ["<=50K", ">50K"])

,<=50K,>50K
<=50K,131,7358
>50K,9,2299


In [45]:
1 - qda_clf.score(X[~train], y[~train])

0.7519648872103706

**f) Perform logistic regression on the training data in order to predict income using the
variables that seemed most associated with income in (b). What is the test error of
the model obtained?**

In [46]:
# By default scikit-learn uses an l2 regularization penalty, which we don't want for
# vanilla logistic regression as described in the book
log_reg = LogisticRegression(penalty = "none", solver = "lbfgs")
log_reg.fit(X[train], y[train])

LogisticRegression(penalty='none')

In [47]:
log_reg_pred = log_reg.predict(X[~train])
pd.DataFrame(confusion_matrix(y[~train], log_reg_pred), 
             columns = ["<=50K", ">50K"], index = ["<=50K", ">50K"])

,<=50K,>50K
<=50K,7253,236
>50K,1722,586


In [48]:
log_reg.score(X[~train], y[~train])

0.800142900888027

**g) Perform KNN on the training data, with several values of K, in order to predict
income. Use only the variables that seemed most associated with income in (b).
What test errors do you obtain? Which value of K seems to perform the best on this
data set?**

In [56]:
#create new a knn model
knn = KNeighborsClassifier()

#create a dictionary of all values we want to test for n_neighbors
param_grid = {'n_neighbors': np.arange(1, 25)}

#use gridsearch to test all values for n_neighbors
knn_gscv = GridSearchCV(knn, param_grid, cv=5)

#fit model to data
knn_gscv.fit(X[train], y[train])

#check top performing n_neighbors value
knn_gscv.best_params_

{'n_neighbors': 1}

In [61]:
knn_gscv.best_estimator_
knn_pred = knn_gscv.best_estimator_.predict(X[~train])
pd.DataFrame(confusion_matrix(y[~train], knn_pred), 
             columns = ["<=50K", ">50K"], index = ["<=50K", ">50K"])
knn_gscv.best_estimator_.score(X[~train], y[~train])

AttributeError: 'Flags' object has no attribute 'c_contiguous'

**h) Choose which model predicts income the best and justify your choice.**

KNN didn't work, so I conclude that Logistic Regression is best at predicting income. 